In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Loan-Prediction-System'

In [2]:
from dataclasses import dataclass
from pathlib import Path

# enitity
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    encoder_name: str
    test_size: float
    all_cols: list

In [3]:
from sentimentAnalysis.constants import *
from sentimentAnalysis.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        all_cols = list(self.schema.COLUMNS.keys())

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            encoder_name=config.encoder_name,
            test_size=config.test_size,
            all_cols=all_cols,
        )

        return data_transformation_config

In [5]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from loanPrediction.utils.common import save_bin
from pathlib import Path
from sklearn.model_selection import train_test_split  
from loanPrediction import logger

In [6]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def transform_data(self):
        data = pd.read_csv(self.config.data_path)
        
        # take only needed cols:
        data = data[self.config.all_cols]

        # impute categorical data
        cols = data[["Gender", "Married", "Self_Employed", "Education", "Credit_History", "Property_Area"]] 
        for  i in cols: 
            data[i].fillna(data[i].mode().iloc[0], inplace=True)

        # impute the numerical data
        n_cols = data[["LoanAmount", "Loan_Amount_Term", "CoapplicantIncome", "ApplicantIncome"]] 
        for i in n_cols: 
            data[i].fillna(data[i].mean(axis=0), inplace=True)

        # drop rows don't have target
        if data.isna().any().any():
            data.dropna(inplace=True)

        # Transform data
        object_cols = ["Gender",'Married','Education','Self_Employed','Property_Area','Loan_Status']
        encoder = OrdinalEncoder()
        data[object_cols] = encoder.fit_transform(data[object_cols])
        save_bin(data=encoder, path=Path(os.path.join(self.config.root_dir, self.config.encoder_name)))
        data.to_csv(os.path.join(self.config.root_dir, "encoded_data.csv"), index=False)
        

    def split_data(self):
        data = pd.read_csv(os.path.join(self.config.root_dir, "encoded_data.csv"))
        
        X = data.drop("Loan_Status", axis=1) 
        y = data["Loan_Status"] 

        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=2) 

        train = pd.concat([X_train, y_train], axis=1)
        test = pd.concat([X_test, y_test], axis=1)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_data()
    data_transformation.split_data()
except Exception as e:
    raise e

[2023-12-11 13:20:48,655: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-11 13:20:48,659: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-11 13:20:48,663: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-11 13:20:48,664: INFO: common: created directory at: artifacts]
[2023-12-11 13:20:48,665: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-11 13:20:48,680: INFO: common: binary file saved at: artifacts/data_transformation/encoder.joblib]
[2023-12-11 13:20:48,704: INFO: 421386631: Splited data into training and test sets]
[2023-12-11 13:20:48,706: INFO: 421386631: (491, 11)]
[2023-12-11 13:20:48,706: INFO: 421386631: (123, 11)]
